In [2]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.metrics import precision_score
from datasets import Dataset, load_dataset
from pandas.core.interchange.dataframe_protocol import DataFrame
from transformers import TrainingArguments, Trainer
# Step 1: Install required libraries

from transformers import ElectraForSequenceClassification, ElectraTokenizerFast
from sklearn.model_selection import train_test_split
import pandas as pd

# Step 1: load dataset
df = pd.read_csv(f"../dataset/preprocess/github-labels-top3-803k-train_0.1percent_train.csv")
df = df.rename(columns={"issue_label": "label", "issue_data": "text"})
# df = pd.concat([df.text, df.label], axis=1)
pd.set_option('future.no_silent_downcasting', True)
df['label'] = df['label'].replace(['enhancement', 'bug', 'question'],[0,1,2])
# print(df)

dataset = Dataset.from_pandas(df)

# dataset = load_dataset('ag_news')

tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator")

def preprocess(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Step 2: split into train and test
# print(dataset)
split_data = dataset.train_test_split(test_size=0.2)
train_df = split_data['train']
test_df = split_data['test']

# train_df = dataset['train']
# test_df = dataset['test']

# print(train_df)
# print(test_df)


# print(tokens)

# inputs = tokenizer(
#     train_df.issue_data.values.tolist(),
#     padding="max_length",
#     truncation=True,
#     return_tensors="pt",         # Return PyTorch tensors
#     return_attention_mask=True,  # Include attention mask
# )
# print(inputs)
# return tokenizer(batch, padding="max_length", truncation=True, max_length=256)
# inputs = tokenizer(train_df.issue_data.tolist())

model = ElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator")

# outputs = model(
#     input_ids=inputs["input_ids"],
#     attention_mask=inputs["attention_mask"]
# )


# Define training args
training_args = TrainingArguments(
    output_dir="../dataset",
    # label_names=["issue_label"],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=4,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",  # Disable WandB unless needed
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted"),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=test_df,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# save model
# trainer.save_model("trained_electra_model")

# save tokenizer
# tokenizer.save_pretrained("")


Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.510700,0.511945,0.692308,0.637363,0.692308,0.661123
2,0.337100,0.327291,0.769231,0.693055,0.769231,0.728740
3,0.276600,0.302783,0.807692,0.734517,0.807692,0.768429
4,0.063200,0.369580,0.788462,0.714494,0.788462,0.749475


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Framew

TrainOutput(global_step=208, training_loss=0.3360430947863139, metrics={'train_runtime': 42.6971, 'train_samples_per_second': 38.972, 'train_steps_per_second': 4.872, 'total_flos': 109454199029760.0, 'train_loss': 0.3360430947863139, 'epoch': 4.0})